## Detecção de Argumentos Faltantes em Artigos da Wikipedia com Gemini API

Este notebook demonstra como usar a API Gemini (via `google-generativeai`) para analisar seções de um artigo da Wikipedia e identificar argumentos ou contra-argumentos importantes que podem estar faltando.

### 1. Imports, Configuração da API e Constantes

In [2]:
# Imports principais
import os
import json
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    try:
        genai.configure(api_key=api_key)
        print("API Gemini configurada com sucesso.")
    except Exception as e:
        print(f"Erro ao configurar a API Gemini: {e}")
        api_key = None # Garante que não tentaremos usar uma configuração falha

# Constantes do Modelo
DEFAULT_MODEL_NAME = "gemini-1.5-flash-latest"
DEFAULT_TEMPERATURE = 0.3

API Gemini configurada com sucesso.


### 2. Função Auxiliar para Parse da Resposta JSON do LLM

In [38]:
def _parse_llm_json_response(response_text: str) -> list[dict] | dict:
    """
    Tenta analisar a string de resposta JSON do LLM.
    Remove possíveis blocos de código Markdown e, se a análise direta falhar,
    tenta um fallback analisando linha por linha objetos JSON.
    Retorna uma lista de dicionários, um dicionário, ou uma lista vazia em caso de falha.
    """
    if not response_text:
        return []

    cleaned_text = response_text.strip()
    
    # Remove ```json ... ``` ou ``` ... ``` se presentes
    if cleaned_text.startswith("```json"):
        cleaned_text = cleaned_text[7:]
        if cleaned_text.endswith("```"):
            cleaned_text = cleaned_text[:-3]
    elif cleaned_text.startswith("```") and cleaned_text.endswith("```"):
        cleaned_text = cleaned_text[3:-3]
            
    cleaned_text = cleaned_text.strip()

    try:
        # Tenta carregar o JSON diretamente
        return json.loads(cleaned_text)
    
    except json.JSONDecodeError:
        # print(f"DEBUG: JSONDecodeError na resposta completa. Tentando fallback linha por linha. Resposta:\n{cleaned_text}") # Para debug
        items = []
        for line in cleaned_text.splitlines():
            line = line.strip()
            if line.startswith("{") and line.endswith("}"):
                try:
                    # Remove vírgulas no final de uma linha, se houver (comum em listas JSON mal formatadas)
                    if line.endswith(","):
                        line = line[:-1]
                    items.append(json.loads(line))
                except json.JSONDecodeError:
                    # print(f"DEBUG: Não foi possível decodificar a linha JSON: {line}") # Para debug
                    pass # Ignora linhas que não são JSON válidos
            elif line == "[" or line == "]": # Ignora colchetes de lista em linhas separadas
                pass
        
        # Se 'items' contém apenas um dicionário, e o JSON original não parecia ser uma lista, retorna o dicionário.
        if len(items) == 1 and not (cleaned_text.startswith("[") and cleaned_text.endswith("]")):
            if cleaned_text.startswith("{") and cleaned_text.endswith("}"):
                return items[0]
        
        if not items and cleaned_text == "[]": 
            return []
        # print(f"DEBUG: Itens extraídos pelo fallback: {items}") # Para debug
        return items if items else [] # Retorna a lista de itens ou uma lista vazia se nada foi parseado

### 3. Função para Obter Argumentos Faltantes de UMA Seção

In [39]:
def get_missing_args_for_section(section_title: str, section_text: str, article_summary: str | None = None, max_args: int = 2, model_name: str = DEFAULT_MODEL_NAME) -> list[dict]:
    """
    Analisa um trecho de uma seção de artigo da Wikipedia e identifica argumentos faltantes.
    Retorna uma lista de dicionários, cada um com 'argument' e 'priority'.
    """
    context_summary = ""
    if article_summary:
        context_summary = article_summary.strip()
        if len(context_summary) > 600:
            context_summary = context_summary[:597] + "..."           
        
    prompt = f"""
You are an assistant specialized in critical content analysis.
Analyze the following excerpt from a Wikipedia article section and identify up to {max_args} arguments, counter-arguments,
or important viewpoints that appear to be missing or could enrich the discussion about the section's topic.

For each missing item identified, provide:
- "argument": A clear and concise description of the missing argument/point and how it matters for the text understand.
- "priority": An integer from 1 (most important/impactful) to {max_args} (least important among those identified).

Return your response as a pure JSON list of objects.
If no relevant arguments or points are missing, return an empty JSON list: `[]`.

Section Title (for context): "{section_title}"
Section Excerpt:
\"\"\"{section_text.strip()}\"\"\"
"""

    try:
        model_client = genai.GenerativeModel(model_name)
        generation_config = genai.types.GenerationConfig(temperature=DEFAULT_TEMPERATURE, max_output_tokens=256)  
            
        response = model_client.generate_content(prompt, generation_config=generation_config)
        
        response_content = ""
        if response and hasattr(response, 'text') and response.text:
            response_content = response.text
        elif response and response.parts: # Modelos mais novos podem usar 'parts'
             response_content = "".join([part.text for part in response.parts if hasattr(part, 'text')])

        if response_content:
            parsed_response = _parse_llm_json_response(response_content)
            if isinstance(parsed_response, list):
                return parsed_response
            elif isinstance(parsed_response, dict) and parsed_response: # Se retornou um dict único, envolve em lista
                return [parsed_response]
            else:
                return []
        else:
            # print(f"WARN: Resposta inesperada, vazia ou sem texto processável do modelo para seção '{section_title}'.")
            return []

    except Exception as e:
        print(f"ERRO ao chamar a API Gemini para a seção '{section_title}': {e}")
        return []

### 4. Função para Coletar Argumentos Faltantes de Todas as Seções de um Artigo

In [40]:
def get_missing_args_for_article(article: dict, section_col: str = "sections", max_args_per_sec: int = 2) -> dict:
    """
    Recebe um dicionário de artigo com uma lista de seções e 
    retorna um dicionário onde as chaves são títulos de seção e os valores são listas 
    de argumentos faltantes (cada um sendo um dict com 'argument' e 'priority').
    Ex: { 'Introdução': [{'argument': '...', 'priority': 1}], ... }
    """
    
    missing_by_section = {}
    if section_col not in article or not isinstance(article.get(section_col), list):
        print(f"AVISO: Chave '{section_col}' não encontrada no artigo ou não é uma lista.")
        return missing_by_section

    for sec in article.get(section_col, []):
        if not isinstance(sec, dict) or "title" not in sec or "content" not in sec:
            print(f"AVISO: Seção ignorada por não ter 'title' ou 'content': {sec}")
            continue
        
        section_title = sec["title"]
        section_content = sec["content"]
        
        print(f"Processando seção: {section_title}...")
        args = get_missing_args_for_section(
            section_title=section_title,
            section_text=section_content,
            max_args=max_args_per_sec
        )
        if args: # Adiciona apenas se a lista de argumentos não estiver vazia
            missing_by_section[section_title] = args
        else:
            print(f"Nenhum argumento faltante encontrado para a seção: {section_title}")
    return missing_by_section

### 5. Função para Sumarizar e Rankear os Principais Argumentos Faltantes do Artigo

In [41]:
def summarize_missing_args_for_article(
    missing_args_by_section: dict, # Dicionário retornado por get_missing_args_for_article
    max_summary_items: int = 5,
    model_name: str = DEFAULT_MODEL_NAME
) -> list[dict]:
    """
    A partir do dicionário {seção: [lista de args faltantes]}, agrupa todos os argumentos,
    remove duplicatas ou itens muito semelhantes, e rankeia os `max_summary_items` principais.
    Retorna uma lista de dicionários, cada um com 'argument', 'section' (origem), e 'priority'.
    """
    
    if not api_key:
        return []
        
    if not missing_args_by_section:
        return []

    # 1) Flatten: Cria uma lista única de todos os argumentos faltantes com suas seções e prioridades
    all_missing_entries = []
    for section, args_list in missing_args_by_section.items():
        if isinstance(args_list, list): # Garante que args_list é uma lista
            for arg_item in args_list:
                if isinstance(arg_item, dict) and "argument" in arg_item and "priority" in arg_item:
                    all_missing_entries.append({
                        "section": section,
                        "argument": arg_item["argument"],
                        "priority": arg_item["priority"]
                    })
                # else: print(f"DEBUG: Item de argumento ignorado por formato inválido: {arg_item}")
        # else: print(f"DEBUG: args_list para seção '{section}' não é uma lista: {args_list}")
    
    if not all_missing_entries:
        print("Nenhum argumento válido encontrado após o flatten para sumarização.")
        return []

    # 2) Monta o prompt para o LLM, pedindo para consolidar e rankear
    prompt_lines = []
    for entry in all_missing_entries:
        prompt_lines.append(f"- Da seção '{entry['section']}' (prioridade original {entry['priority']}): "
                            f"{entry['argument']}")
    
    input_arguments_str = "\n".join(prompt_lines)

    prompt = f"""
You are a senior research assistant and editor.
The list below contains suggestions for missing arguments or points identified in different sections of a Wikipedia article.
Your task is to consolidate this list by:
1. Removing any arguments that are duplicated or semantically very similar to each other, keeping the clearer or more comprehensive version.
2. Based on relevance and potential impact for article completeness, select the {max_summary_items} most important missing arguments.
3. For each of the {max_summary_items} selected, maintain the information about the source section and original priority.

Return your response as a pure JSON list of objects. Each object should have the fields:
- "argument": The text of the consolidated missing argument.
- "section": The name of the section where the argument was originally identified.

If the input list is empty or contains no valid arguments, return an empty JSON list: `[]`.

List of missing arguments for analysis:
{input_arguments_str}
"""

    try:
        model_client = genai.GenerativeModel(model_name)
        generation_config = genai.types.GenerationConfig(temperature=DEFAULT_TEMPERATURE)
        
        # print(f"DEBUG: Enviando prompt para sumarização:\n{prompt[:500]}...")
        response = model_client.generate_content(prompt, generation_config=generation_config)
        
        # print(f"DEBUG: Resposta crua da API para sumarização:\n{response.text if hasattr(response, 'text') else response}")
        
        response_content = ""
        if response and hasattr(response, 'text') and response.text:
            response_content = response.text
        elif response and response.parts:
             response_content = "".join([part.text for part in response.parts if hasattr(part, 'text')])

        if response_content:
            parsed_response = _parse_llm_json_response(response_content)
            if isinstance(parsed_response, list):
                return parsed_response
            elif isinstance(parsed_response, dict) and parsed_response:
                return [parsed_response]
            else:
                # print(f"WARN: Resposta JSON da sumarização não era uma lista. Resposta: {parsed_response}")
                return []
        else:
            # print("WARN: Resposta inesperada, vazia ou sem texto processável do modelo para sumarização.")
            return []

    except Exception as e:
        print(f"ERRO ao chamar a API Gemini para sumarização: {e}")
        return []

### 6. Exemplo de Artigo e Execução do Pipeline

In [42]:
# Artigo fictício sobre IA com omissões deliberadas para teste
mock_article = {
    "id": 101010,
    "title": "Artificial Intelligence: Fundamentals and Applications",
    "sections": [
        {
            "title": "Introduction to AI",
            "content": (
                "Artificial Intelligence (AI) is a branch of computer science "
                "dedicated to creating systems capable of performing tasks that normally "
                "would require human intelligence. This includes, but is not limited to, natural language "
                "processing, complex pattern recognition, and autonomous decision-making."
            )
        },
        {
            "title": "Brief History of AI",
            "content": (
                "The formal concept of AI can be traced back to Alan Turing's pioneering work in the 1950s, "
                "particularly with the proposal of the Turing Test as a way to evaluate machine intelligence. "
                "Later, between the 1960s and 1970s, the first expert systems emerged, "
                "which used knowledge bases and rules encoded by experts to solve "
                "problems in specific domains. After a period of reduced investment and progress, known as the 'AI winter', "
                "the field experienced a significant resurgence from the 1990s, driven mainly "
                "by significant advances in computational capacity and the availability of large volumes of data."
            )
        },
        {
            "title": "Current Main AI Techniques",
            "content": (
                "Currently, the AI field is heavily influenced by machine learning algorithms, "
                "with emphasis on deep learning. Convolutional Neural Networks (CNNs) are "
                "widely used in computer vision tasks, such as image and video recognition, "
                "while architectures like Recurrent Neural Networks (RNNs) and, more prominently, Transformers "
                "dominate natural language processing applications, such as machine translation and text generation."
            )
        },
        {
            "title": "Examples of AI Applications",
            "content": (
                "The commercial, industrial, and scientific applications of Artificial Intelligence are vast and include:\n"
                "- Computer-assisted medical diagnosis, medical image analysis, and drug discovery.\n"
                "- Personalized recommendation systems in e-commerce platforms, streaming, and social networks.\n"
                "- Development of autonomous vehicles, logistics route optimization, and intelligent traffic management.\n"
                "- Financial data analysis for fraud detection, market trend prediction, and algorithmic trading."
            )
        }
        # Intentional note: This article deliberately omits crucial discussions about:
        # - AI Ethics (e.g., algorithmic bias, privacy, responsibility)
        # - Social and economic impact of AI (e.g., job market, inequality)
        # - AI model security and robustness (e.g., adversarial attacks, explainability)
        # - AI alignment issues (ensuring advanced AIs act according to human values)
        # - Current AI limitations and future research challenges.
    ]
}

# Execução do pipeline
if api_key:
    # Verifica se o modelo padrão é válido antes de prosseguir
    model_is_valid = False
    try:
        genai.get_model(f"models/{DEFAULT_MODEL_NAME}") # A API espera o prefixo 'models/'
        model_is_valid = True
        print(f"Modelo '{DEFAULT_MODEL_NAME}' verificado com sucesso.")
    except Exception as e:
        print(f"ERRO ao verificar o modelo '{DEFAULT_MODEL_NAME}': {e}")
        print("Verifique se o nome do modelo está correto e se você tem acesso a ele.")

    if model_is_valid:
        print("\n--- Iniciando análise de argumentos faltantes no artigo ---")
        missing_args_all_sections = get_missing_args_for_article(mock_article, max_args_per_sec=2)
        
        print("\n--- Argumentos faltantes identificados por seção ---")
        if missing_args_all_sections:
            for section, args in missing_args_all_sections.items():
                print(f"\nSeção: {section}")
                if args: # Verifica se a lista de args não é None ou vazia
                    for arg_item in args:
                        print(f"  - Prioridade {arg_item.get('priority', 'N/A')}: {arg_item.get('argument', 'Argumento não especificado')}")
                else:
                    print("  Nenhum argumento faltante retornado para esta seção.")
        else:
            print("Nenhum argumento faltante identificado em nenhuma seção.")

        print("\n--- Sumarizando os principais argumentos faltantes do artigo ---")
        summary_top_missing = summarize_missing_args_for_article(missing_args_all_sections, max_summary_items=5)

        if summary_top_missing:
            print("\nPrincipais argumentos faltantes no artigo (sumarizado):")
            for item in summary_top_missing:
                print(f"  - Seção '{item.get('section', 'N/A')}': {item.get('argument', 'N/A')} (Prioridade Original: {item.get('priority', 'N/A')})")
        else:
            print("Nenhum argumento faltante para sumarizar ou a sumarização falhou.")
    else:
        print("\nExecução do pipeline principal pulada devido a erro na verificação do modelo.")
else:
    print("\nExecução do pipeline principal pulada. Verifique a configuração da GENAI_API_KEY.")

Modelo 'gemini-1.5-flash-latest' verificado com sucesso.

--- Iniciando análise de argumentos faltantes no artigo ---
Processando seção: Introduction to AI...
Processando seção: Brief History of AI...
Processando seção: Current Main AI Techniques...
Processando seção: Examples of AI Applications...

--- Argumentos faltantes identificados por seção ---

Seção: Introduction to AI
  - Prioridade 1: The ethical implications of AI development and deployment are largely absent.  The excerpt focuses solely on the technical capabilities of AI, neglecting crucial discussions around bias in algorithms, job displacement due to automation, the potential for misuse in surveillance or autonomous weapons systems, and the broader societal impact of increasingly intelligent machines.  A balanced introduction to AI should acknowledge these concerns.
  - Prioridade 2: Different approaches and paradigms within AI are not mentioned. The excerpt presents a very general definition.  A richer introduction wou